In [703]:
import torch
import sys
sys.path.append('../gcn')
import pandas as pd
from model import TwoLayerGCN
from trainer import Trainer, RunConfig
from utils import Dataset, load_data
from torch_geometric.data import Data
import numpy as np
import igraph
import networkx as nx
import torch_geometric 
import torch_geometric.data as torch_data

In [704]:
features, labels, train_labels, val_labels, test_labels, adjacency_matrix, \
    laplacian_matrix, num_classes = load_data(Dataset.Cora)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# training parameters
run_config = RunConfig(learning_rate=0.1, num_epochs=200, weight_decay=5e-4, output_dir="gcn/")

# constructing a GCN model
model = TwoLayerGCN(
        input_size=features.size(1),
        hidden_size=16,
        output_size=num_classes,
        dropout=0.5
    )

In [705]:
features_friends = pd.read_csv("../data/FINAL_FEATURES_FRIENDS.tsv", sep="\t")
target_traintest = pd.read_csv("../data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep="\t")
sequences_matrix = pd.read_csv("../data/FINAL_SEQUENCES_MATRIX.tsv", sep="\t")
targets_traintest = pd.read_csv("../data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep="\t")

In [706]:
targets_train=targets_traintest.loc[targets_traintest.TARGET!='test']

In [707]:
temp_df = target_traintest.merge(features_friends, on='CLIENT_ID', how = 'left')

In [708]:
tmp_df = df_chunk.merge(target_traintest, on='CLIENT_ID', how = 'left')

In [709]:
tmp_df_test = tmp_df[tmp_df['TARGET'] == 'test']
tmp_df = tmp_df[tmp_df['TARGET'] != 'test']

In [710]:
tmp_df.drop(['FRIEND_ID','cluster','RETRO_DT','TARGET'], axis=1)

,CLIENT_ID,i1047,i1048,i1056,i1058,i1059,i1060,i1065,i1066,i1067,...,u8=17,u8=18,u8=19,u8=2,u8=20,u8=21,u8=6,u8=7,u8=8,u8=9
0,999819,0.0,0.0,8.0,1.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,999819,0.0,45.0,41.0,0.0,18.0,2.0,17.0,0.0,6.0,...,0.0,0.0,29.0,0.0,0.0,133.0,0.0,0.0,0.0,0.0
2,999819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,999819,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,999819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,99607,0.0,197.0,11.0,3.0,0.0,9.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,99607,0.0,0.0,49.0,0.0,0.0,1.0,49.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,99607,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
998,99607,0.0,6.0,66.0,2.0,1.0,10.0,28.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [711]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [712]:
chunksize = 1000
for df_chunk in pd.read_csv('../data/FINAL_FEATURES_FRIENDS.tsv', chunksize=chunksize, sep='\t'):
    print(len(df_chunk))
    break
features_friends.head()

1000


,CLIENT_ID,i1047,i1048,i1056,i1058,i1059,i1060,i1065,i1066,i1067,...,u8=18,u8=19,u8=2,u8=20,u8=21,u8=6,u8=7,u8=8,u8=9,FRIEND_ID
0,999819,0.0,0.0,8.0,1.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99981973.0
1,999819,0.0,45.0,41.0,0.0,18.0,2.0,17.0,0.0,6.0,...,0.0,29.0,0.0,0.0,133.0,0.0,0.0,0.0,0.0,99981949.0
2,999819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99981979.0
3,999819,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99981939.0
4,999819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99981975.0


In [713]:
# df_chunk = df_chunk.drop(df_chunk.columns[df_chunk.columns.str.startswith('u')], axis=1)

In [714]:
df_chunk['cluster'] = [str(int(x % 2 == 0)) for x in range(len(df_chunk))]

In [715]:
G = nx.from_pandas_edgelist(tmp_df, 'CLIENT_ID', 'FRIEND_ID', edge_attr=['cluster'])

In [716]:
tmp_df['TARGET']

0      0
1      0
2      0
3      0
4      0
      ..
995    1
996    1
997    1
998    1
999    1
Name: TARGET, Length: 817, dtype: object

In [717]:
tmp_df.drop(['CLIENT_ID','TARGET','FRIEND_ID','cluster','RETRO_DT'],axis=1)

,i1047,i1048,i1056,i1058,i1059,i1060,i1065,i1066,i1067,i1068,...,u8=17,u8=18,u8=19,u8=2,u8=20,u8=21,u8=6,u8=7,u8=8,u8=9
0,0.0,0.0,8.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,45.0,41.0,0.0,18.0,2.0,17.0,0.0,6.0,6.0,...,0.0,0.0,29.0,0.0,0.0,133.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,197.0,11.0,3.0,0.0,9.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,49.0,0.0,0.0,1.0,49.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
998,0.0,6.0,66.0,2.0,1.0,10.0,28.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [718]:
tmp_df['TARGET'].astype(int)

0      0
1      0
2      0
3      0
4      0
      ..
995    1
996    1
997    1
998    1
999    1
Name: TARGET, Length: 817, dtype: int64

In [719]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(tmp_df.drop(['CLIENT_ID','TARGET','FRIEND_ID','cluster','RETRO_DT'],axis=1), tmp_df['TARGET'].astype(int))

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [720]:
X_test = tmp_df_test.drop(['CLIENT_ID','TARGET','FRIEND_ID','cluster','RETRO_DT'],axis=1)
y_test = tmp_df_test['TARGET']

In [721]:
y_pred = model.predict(X_test)

In [725]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1])

In [723]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

TypeError: Labels in y_true and y_pred should be of the same type. Got y_true=['test'] and y_pred=[0 1]. Make sure that the predictions provided by the classifier coincides with the true labels.

In [ ]:
tmp_features = df_chunk.drop(['CLIENT_ID','FRIEND_ID','cluster'], axis=1)

In [ ]:
features_pt = torch.from_numpy(tmp_chunk.values)

In [ ]:
# corr_features = corr_features.iloc[1:]
# corr_features = corr_features.drop(['CLIENT_ID','FRIEND_ID'], axis = 1)

In [ ]:
# corr_features = corr_features.iloc[:-1]

In [ ]:
# corr_features

In [ ]:
import networkx as nx

from bokeh.io import output_file, show
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
from bokeh.io import output_notebook
output_notebook()

cluser_colors = {'0': 'red', '1': 'black'}
edge_attrs = {}
for start_node, end_node, attr in G.edges(data=True):
    edge_attrs[(start_node, end_node)] = cluser_colors[attr['cluster']]
nx.set_edge_attributes(G, edge_attrs, "edge_color")
plot = Plot(width=400, height=400,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "VK Graph"

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("cluster", "@cluster")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

# output_file("interactive_graphs.html")
show(plot)

In [ ]:
pyg_graph = torch_geometric.utils.from_networkx(G)

In [ ]:
tmp_df_features = tmp_df.drop(['CLIENT_ID', 'FRIEND_ID', 'cluster', 'RETRO_DT', 'TARGET'], axis = 1)

In [ ]:
tmp_df_features

In [ ]:
X = torch.from_numpy(tmp_df_features.values)

In [ ]:
X.shape

In [ ]:
# target_val=pd.DataFrame()
# for i in range(len(ess_target)):
#     target_val['TARGET'].iloc[i] = int(ess_target[i]['TARGET'])
#     # print(ess_target[i]['TARGET'])
#     target_val.loc['CLIENT_ID', i] = int(ess_target[i]['CLIENT_ID'])

In [ ]:
# target_val.astype(int).transpose()['TARGET'].value_counts()

In [ ]:
df_chunk

In [ ]:
train_labels_pt = torch.tensor(tmp_df['CLIENT_ID'], dtype=torch.float)
train_labels_pt.shape

In [ ]:
val_labels_pt = torch.tensor(tmp_df['TARGET'].astype(int), dtype=torch.float)
val_labels_pt.shape

In [ ]:
# target_val.loc['TARGET']

In [ ]:
adjacency_matrix_pt = torch_data.Data(np.matrix(nx.adjacency_matrix(G).todense()), dtype=torch.float)

In [ ]:
adjacency_matrix_pt

In [ ]:
# corr_feature_pt=torch.tensor(corr_features.astype(int), dtype=torch.float)

In [ ]:
val_labels_pt.shape

In [ ]:
# training
trainer = Trainer(model)
trainer.train(X, train_labels_pt, val_labels_pt, adjacency_matrix_pt, device, run_config, log=False)

In [ ]:
class GCNConv(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNConv, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=False)

    def forward(self, x: torch.Tensor, adjacency_hat: torch.sparse_coo_tensor):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency_hat, x)
        return x

In [ ]:
import torch.nn as nn
class TwoLayerGCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, adj_matrix, dropout = 0.1):
        super(TwoLayerGCN, self).__init__()
        self.adj_matrix = adj_matrix
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor,  labels: torch.Tensor = None):
        x = self.dropout(x)
        x = self.conv1(x, self.adj_matrix)
        print(x.shape)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, self.adj_matrix)

        if labels is None:
            return x

        loss = nn.CrossEntropyLoss()(x, labels)
        return x, loss

In [ ]:
X_inp = X[0]

In [ ]:
adjacency_matrix_pt

In [ ]:
model = TwoLayerGCN(input_size=features.size(1),
        hidden_size=16,
        output_size=num_classes,
        dropout=0.5, adj_matrix = adjacency_matrix_pt)

In [ ]:
class SocialGNN(torch.nn.Module):
    def __init__(self,num_of_feat,f):
        super(SocialGNN, self).__init__()
        self.conv1 = GCNConv(num_of_feat, f)
        self.conv2 = GCNConv(f, 2)

    def forward(self, data):
        x = data.x.float()
        edge_index =  data.edge_index
        x = self.conv1(x=x, edge_index=edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
G.nodes

In [ ]:
adjacency_matrix_pt = torch_data.Data(np.matrix(nx.adjacency_matrix(G).todense()), dtype=torch.float)

In [ ]:
adjacency_matrix_pt

In [ ]:
# evaluating
ce_loss, accuracy = trainer.evaluate(features, test_labels, adjacency_matrix, device)